# local_tts_v2 — Quick Test
Hexagonal architecture smoke test: normalisation → synthesis → file output.

In [ ]:
import sys
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for candidate in [start.resolve()] + list(start.resolve().parents):
        if (candidate / 'src' / 'tts_v2').exists():
            return candidate
    raise RuntimeError('Could not locate local_tts_v2 repo root')

REPO_ROOT = find_repo_root(Path.cwd())
if str(REPO_ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / 'src'))

OUTPUT_DIR = REPO_ROOT / 'outputs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f'Repo root : {REPO_ROOT}')
print(f'Output dir: {OUTPUT_DIR}')

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(name)s | %(levelname)s | %(message)s')

# Domain + service
from tts_v2.domain.audio import SynthesisRequest
from tts_v2.domain.voice import list_personas
from tts_v2.service.tts_service import TTSService

# Adapters
from tts_v2.adapters.synthesizer.coqui_adapter import CoquiSynthesizerAdapter
from tts_v2.adapters.normalizer.bfsi_normalizer_adapter import BFSINormalizerAdapter
from tts_v2.adapters.audio_sink.file_sink_adapter import FileSinkAdapter
from tts_v2.adapters.audit.file_audit_adapter import FileAuditAdapter

print('✓ Imports successful')
print(f'Available personas: {list_personas()}')

## Normalisation Smoke Tests

In [ ]:
from tts_v2.adapters.normalizer.bfsi_normalizer_adapter import BFSINormalizerAdapter
norm = BFSINormalizerAdapter()

tests = [
    ('abbreviation', 'Please complete your KYC verification with NAB.'),
    ('money',        'Your account balance is $1,234.50.'),
    ('otp',          'Your OTP is 482913. Do not share it.'),
]

for label, raw in tests:
    result = norm.normalize(raw)
    print(f'[{label}]')
    print(f'  IN : {raw}')
    print(f'  OUT: {result}')
    print()

## Wire TTSService and Load Model

In [ ]:
service = TTSService(
    synthesizer=CoquiSynthesizerAdapter(use_gpu=True),
    normalizer=BFSINormalizerAdapter(),
    audio_sink=FileSinkAdapter(),
    audit=FileAuditAdapter(str(OUTPUT_DIR / 'audit.jsonl')),
)
print('✓ TTSService ready')

## Synthesise BFSI Scenarios

In [ ]:
scenarios = [
    {
        'name'   : 'balance_inquiry',
        'text'   : 'Your account balance is $1,234.50 as of today.',
        'persona': 'professional_female',
    },
    {
        'name'   : 'otp_delivery',
        'text'   : 'Your OTP is 482913. This code expires in 5 minutes.',
        'persona': 'neutral_male',
    },
    {
        'name'   : 'kyc_notice',
        'text'   : 'We require KYC verification to continue. Please contact NAB.',
        'persona': 'professional_male',
    },
]

results = []
for s in scenarios:
    out_path = str(OUTPUT_DIR / f"{s['name']}.wav")
    result = service.speak(
        SynthesisRequest(
            text=s['text'],
            persona=s['persona'],
            output_path=out_path,
        )
    )
    results.append((s['name'], result))
    status = '✓' if result.success else '✗'
    print(f'{status} {s["name"]} → {result.output_path}')

## Verify Output Files

In [ ]:
import IPython.display as ipd

for name, result in results:
    p = Path(result.output_path)
    size_kb = p.stat().st_size / 1024 if p.exists() else 0
    print(f'{name}: {size_kb:.1f} KB — {"exists ✓" if p.exists() else "MISSING ✗"}')
    if p.exists():
        display(ipd.Audio(str(p)))